# 0. Initial Set-up

In [ ]:
import numpy as np
import pandas as pd
import requests
import os
import json
import tqdm

from rdkit import Chem
from tqdm import tqdm
from thermo import functional_groups
from Bio import Entrez
from chembl_structure_pipeline import checker
from rdkit.Chem import rdMolDescriptors, Descriptors, Lipinski, Crippen, inchi
from rdkit.Chem.FilterCatalog import FilterCatalog, FilterCatalogParams
from concurrent.futures import ThreadPoolExecutor, as_completed

import utils
import filters
from data_gathering import download_and_save

[06:33:37] Initializing Normalizer


In [ ]:
# Save the path to your curation folder to "curation_path"
dataset_name = 'AID435034'
curation_path = 'S:/coding/WelQrate/' 
data_folder = f'{curation_path}/data/{dataset_name}'

# Columns to be extracted from the assays:
# Modify if your datasets have different format
smi_col = 'PUBCHEM_EXT_DATASOURCE_SMILES' # Column containing SMILES
cid_col = 'PUBCHEM_CID' # Column containing identifiers (e.g, CIDs)
activity_col = 'PUBCHEM_ACTIVITY_OUTCOME' # Column containing activity outcomes
col_list = [cid_col, smi_col, activity_col]

# 1. Data gathering

Before importing data, need to identify which AIDs will be included. 

Data will be imported from https://pubchem.ncbi.nlm.nih.gov/assay/. For more information on PubChem's programmatic access, refer to: https://pubchem.ncbi.nlm.nih.gov/docs/bioassays. Some other programmatic access options available such as PUG-REST. However, these might not be optimal for bulk retrieval or handling of large dataset due to the limitation of request volume.

Data for individual assays include 7 required columns (CIDs, isomeric SMILES, etc.) and optional test results. Refer to https://ftp.ncbi.nlm.nih.gov/pubchem/Bioassay/CSV/README for further details. For datasets intended for regression model, additional columns could be extracted accordingly.

In [ ]:
# Desired AIDs:
AIDs = [628, 677, 859, 860]

In [ ]:
#Keep unique values in list AIDs (since there could be overlapping AIDs from different targets or project)
AIDs = list(set(AIDs))
AIDs = [str(AID) for AID in AIDs]
print('Number of datasets retrieving: ', len(AIDs))

Number of datasets retrieving:  4


In [ ]:
download_and_save(AIDs, data_folder, col_list, smi_col, cid_col, activity_col)

1 out of 4 complete
2 out of 4 complete
3 out of 4 complete
4 out of 4 complete


# 2. Isomeric SMILES

For the purpose of our project, we would like to include isomeric form of SMILES representation in our final dataset. Although PubChem claimed that their datatable should include isomeric SMILES (https://pubchem.ncbi.nlm.nih.gov/docs/bioassays), some dataset might include non-isomeric SMILES. This step is to import isomeric SMILES based on CIDs.

Several packages such as RDkit have modules to return isomeric SMILES from a given input SMILES. However, for consistency, we decided to use the PubChem Identifier Exchange Service, which take an input identifier (CIDs, SMILES, InChI, etc.)  and return the corresponding identifier (CIDs, isomeric SMILES, InChIs, etc.). Here, we export the list of CIDs for compounds in our dataset and use this server to retrieve their isomeric SMILES. For more information, refer to: https://pubchem.ncbi.nlm.nih.gov/docs/identifier-exchange-service

In [ ]:
#Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_2'):
    os.makedirs(f'{data_folder}/before_finished/step_2')

#Export list of CIDs to csv with one column (without the column name):
for AID in AIDs:
    assay = pd.read_csv(f'{data_folder}/before_finished/step_1/AID{AID}.csv')
    cids = assay['PUBCHEM_CID'].astype(int)  # Ensure the CIDs are integers
    cids.to_csv(f'{data_folder}/before_finished/step_2/CID{AID}.csv', index=False, header=False)

#After this step, we submit the list at https://pubchem.ncbi.nlm.nih.gov/idexchange/idexchange.cgi with operator type "same CID" and Output IDs "SMILES" (isomeric SMILES by default)
#https://pubchem.ncbi.nlm.nih.gov/docs/identifier-exchange-service for more details
#Here I named the output file as "SMILES{AID}.txt"

In [ ]:
def check_isomeric_smiles(AIDs):
    """
    Check if the SMILES in the assay are the same as the isomeric forms returned by pubchem idexchange.
    Input: AIDs (list of strings)
    Output: non_isomeric_smi_cids (dictionary with AID as key and list of CIDs as values for the datasets in AIDs
    """
    non_isomeric_smi_cids = {}
    for AID in AIDs:    
        non_isomeric_smi_cids[AID] = []
        #import SMILES.txt file as a table:
        correct_isomeric_smiles = pd.read_csv(f'{data_folder}/before_finished/step_2/isomeric_smi_{AID}.txt', sep='\t', header=None)
        assay = pd.read_csv(f'{data_folder}/before_finished/step_1/AID{AID}.csv')

        #compare smiles in assay with smiles in correct_smiles:
        for cid in assay['PUBCHEM_CID']:
            if assay.loc[assay['PUBCHEM_CID'] == cid, 'PUBCHEM_EXT_DATASOURCE_SMILES'].values[0] != correct_isomeric_smiles.loc[correct_isomeric_smiles[0] == cid, 1].values[0]:
                non_isomeric_smi_cids[AID].append(cid)

        if len(non_isomeric_smi_cids[AID]) == 0:
            print(f'All SMILES in AID {AID} are isomeric')
        else:
            print(f'There are some potential non-isomeric SMILES in AID {AID}:')
            print(non_isomeric_smi_cids[AID])

    return non_isomeric_smi_cids

def update_isomeric(AIDs, non_isomeric_smi_cids):
    """
    Update the SMILES in the assay to isomeric SMILES.
    Input: AIDs (list of strings), non_isomeric_smi_cids (dictionary with AID as key and list of non-isomeric CIDs as values)
    """
    with open(f'{data_folder}/before_finished/step_2/non_isomeric_smi_cids.txt', 'w') as f:
        # record the non-isomeric SMILES 
        for AID in AIDs:
            assay = pd.read_csv(f'{data_folder}/before_finished/step_1/AID{AID}.csv')
            correct_isomeric_smiles = pd.read_csv(f'{data_folder}/before_finished/step_2/isomeric_smi_{AID}.txt', sep='\t', header=None)
            f.write(f'AID {AID}: {non_isomeric_smi_cids[AID]}\n')

            for cid in non_isomeric_smi_cids[AID]:
                f.write(f'CID {cid}: {assay.loc[assay["PUBCHEM_CID"] == cid, "PUBCHEM_EXT_DATASOURCE_SMILES"].values[0]} -> {correct_isomeric_smiles.loc[correct_isomeric_smiles[0] == cid, 1].values[0]}\n')
                assay.loc[assay['PUBCHEM_CID'] == cid, 'PUBCHEM_EXT_DATASOURCE_SMILES'] = correct_isomeric_smiles.loc[correct_isomeric_smiles[0] == cid, 1].values[0]

            f.write(f'===\n')
            assay.to_csv(f'{data_folder}/before_finished/step_2/AID{AID}.csv', index=False)       

In [ ]:
non_isomeric_smi_cids = check_isomeric_smiles(AIDs)

All SMILES in AID 859 are isomeric
There are some non-isomeric SMILES in AID 628:
[2997662, 2997957, 2999888]
All SMILES in AID 677 are isomeric
All SMILES in AID 860 are isomeric


Note: Here they returned that three smiles in AID628 were not isomeric. This shows that the SMILES representation of some compounds in the given datasets might not be isomeric.

In [ ]:
update_isomeric(AIDs, non_isomeric_smi_cids)

# 3. Import InChIs

We would like to include standard InChI to diversify users' choice of which data they would like to use for their own benchmark.

In [ ]:
# Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_3'):
    os.makedirs(f'{data_folder}/before_finished/step_3')

Again, it is convenient to use the PubChem Identifier Exchange Service (https://pubchem.ncbi.nlm.nih.gov/idexchange/idexchange.cgi) with operator type "same CID" and Output IDs "InChI" to retrieve InChI from a given list of input CIDs. The same CID lists from STEP 2 could be used here. The resulted InChIs could be checked if being standard by indentifying the presence of 'InChI=1S' at the begining of each InChI string.

In [ ]:
"""
CID lists (in "step_2" folder should be submitted to PubChem Identifier Exchange Service)
    Operator type: "same CID" 
    Output IDs "InChI"
    Output method: "Two column file showing each input output-correspondence"
    Compression: "No compression"
InChI list should be saved into "step_3" folder, named as "std_inchi_{AID}.txt" 
"""
# Import dataframes:
for AID in AIDs: 
    exec(f'AID{AID} = pd.read_csv("{data_folder}/before_finished/step_2/AID{AID}.csv")')
    exec(f'AID{AID}_InChI = pd.read_csv("{data_folder}/before_finished/step_3/std_inchi_{AID}.txt", sep="\\t", header=None)')

In [ ]:
#Check if they are all standard InChI:
for AID in AIDs:
    check_inchi = f"""
non_standard_InChI = []
for i in range(len(AID{AID}_InChI[1])):
    if not AID{AID}_InChI[1][i].startswith('InChI=1S'):
        non_standard_InChI.append(AID{AID}_InChI[1][i])
if not non_standard_InChI:
    print('All InChI in AID{AID} are standard')
else:
    print('There are some non-standard InChI in AID{AID}')
    print(non_standard_InChI)
    print('===')
"""
    exec(check_inchi)

All InChI in AID859 are standard
All InChI in AID628 are standard
All InChI in AID677 are standard
All InChI in AID860 are standard


Now we concatenate the InChIs in our tables:

In [ ]:
# Update and save the files
for AID in AIDs: 
    update_inchi = f"""
AID{AID}_InChI_dict = dict(zip(AID{AID}_InChI[0], AID{AID}_InChI[1]))
AID{AID}['InChI'] = AID{AID}[cid_col].map(AID{AID}_InChI_dict)
AID{AID}['InChI'] = AID{AID}['InChI'].astype(str)
AID{AID}[cid_col] = AID{AID}[cid_col].astype(int)
AID{AID}.to_csv(r"{data_folder}/before_finished/step_3/AID{AID}.csv", index=False)
"""
    exec(update_inchi)

# 4. Check duplicates

When checking duplicates in the datasets, we would like to know if there are
1) Multiple identical molecules
2) Molecules with identical CID but different InChIs or SMILES
3) Molecules with identical InChI but with different CIDs or SMILES

In [ ]:
#import:
for AID in AIDs:
    exec(f"AID{AID} = pd.read_csv(r'{data_folder}/before_finished/step_3/AID{AID}.csv', sep=',', header=0)")

#Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_4'):
    os.makedirs(f'{data_folder}/before_finished/step_4')

## 4.1. Checking identical molecules

In [ ]:
#Return all duplicates by comparing InChI, SMILES, and CIDs:
for AID in AIDs:
    check_duplicate = f"""
AID{AID}_duplicates_InChI = AID{AID}[AID{AID}.duplicated(subset=['InChI'], keep=False)]
AID{AID}_duplicates_SMILES = AID{AID}[AID{AID}.duplicated(subset=[smi_col], keep=False)]
AID{AID}_duplicates_CIDs = AID{AID}[AID{AID}.duplicated(subset=[cid_col], keep=False)]
print('Number of AID{AID} InChI duplicates: ', len(AID{AID}_duplicates_InChI))
print('Number of AID{AID} SMILES duplicates: ', len(AID{AID}_duplicates_SMILES))
print('Number of AID{AID} CID duplicates: ', len(AID{AID}_duplicates_CIDs))
"""
    exec(check_duplicate)

Number of AID859 InChI duplicates:  0
Number of AID859 SMILES duplicates:  0
Number of AID859 CID duplicates:  0
Number of AID628 InChI duplicates:  12
Number of AID628 SMILES duplicates:  12
Number of AID628 CID duplicates:  12
Number of AID677 InChI duplicates:  0
Number of AID677 SMILES duplicates:  0
Number of AID677 CID duplicates:  0
Number of AID860 InChI duplicates:  0
Number of AID860 SMILES duplicates:  0
Number of AID860 CID duplicates:  0


In [ ]:
#write duplicates to a txt file: 
with open(f'{data_folder}/before_finished/step_4/duplicates.txt', 'w') as f:
    for AID in AIDs: 
        duplicates_InChI = eval(f'AID{AID}_duplicates_InChI')
        duplicates_SMILES = eval(f'AID{AID}_duplicates_SMILES')
        duplicates_CIDs = eval(f'AID{AID}_duplicates_CIDs')
        f.write(f'\n\nAID{AID} InChI duplicates:\n')
        f.write(duplicates_InChI.to_string())
        f.write(f'\nAID{AID} SMILES duplicates:\n')
        f.write(duplicates_SMILES.to_string())
        f.write(f'\nAID{AID} CID duplicates:\n')
        f.write(duplicates_CIDs.to_string())

## 4.2. Same CIDs but different chemical representations

In [ ]:
#reindex
for AID in AIDs:
    exec(f"AID{AID}_duplicates_CIDs.reset_index(drop=True, inplace=True)")

In [ ]:
with open(f'{data_folder}/before_finished/step_4/sameCID_different_others.txt', 'w') as f:
    for AID in AIDs: 
        sameCID_differentInChI = []
        sameCID_differentSMILES = []
        duplicates_CIDs = eval(f'AID{AID}_duplicates_CIDs')
        for i in range(len(duplicates_CIDs[cid_col])):
            for j in range(i+1, len(duplicates_CIDs[cid_col])):
                if duplicates_CIDs[cid_col][i] == duplicates_CIDs[cid_col][j]:
                    if duplicates_CIDs['InChI'][i] != duplicates_CIDs['InChI'][j]:
                        sameCID_differentInChI.append((duplicates_CIDs[cid_col][i], duplicates_CIDs[cid_col][j]))
                    if duplicates_CIDs[smi_col][i] != duplicates_CIDs[smi_col][j]:
                        sameCID_differentSMILES.append((duplicates_CIDs[cid_col][i], duplicates_CIDs[cid_col][j]))

        if sameCID_differentInChI == []:
            f.write(f'No duplicate CIDs with different InChIs in AID{AID}\n')
        else:
            f.write('Found duplicate CIDs with different InChIs in AID{AID}:\n')
            f.write('\n'.join(str(item) for item in sameCID_differentInChI))
            f.write("\n")
        
        if sameCID_differentSMILES == []:
            f.write(f'No duplicate CIDs with different SMILES in AID{AID}\n')
        else:
            f.write(f'Found duplicate CIDs with different SMILES in AID{AID}:\n')
            f.write('\n'.join(str(item) for item in sameCID_differentSMILES))
            f.write("\n")
        f.write("===\n")

## 4.3. Same InChI but with different CIDs or SMILES

In [ ]:
#reindex
for AID in AIDs:
    exec(f"AID{AID}_duplicates_InChI.reset_index(drop=True, inplace=True)")

In [ ]:
with open(f'{data_folder}/before_finished/step_4/sameInChI_different_others.txt', 'w') as f:
    for AID in AIDs: 
        sameInChI_differentCID = []
        sameInChI_differentSMILES = []
        duplicates_InChI = eval(f'AID{AID}_duplicates_InChI')
        for i in range(len(duplicates_InChI['InChI'])):
            for j in range(i+1, len(duplicates_InChI['InChI'])):
                if duplicates_InChI['InChI'][i] == duplicates_InChI['InChI'][j]:
                    if duplicates_InChI[cid_col][i] != duplicates_InChI[cid_col][j]:
                        sameInChI_differentCID.append((duplicates_InChI[cid_col][i], duplicates_InChI[cid_col][j]))
                    if duplicates_InChI[smi_col][i] != duplicates_InChI[smi_col][j]:
                        sameInChI_differentSMILES.append((duplicates_InChI[cid_col][i], duplicates_InChI[cid_col][j]))
        
        if sameInChI_differentCID == []:
            f.write(f'No duplicate InChIs with different CIDs in AID{AID}\n')
        else:
            f.write('Found duplicate InChIs with different CIDs in AID{AID}:\n')
            f.write('\n'.join(str(item) for item in sameInChI_differentCID))
            f.write("\n")
        
        if sameInChI_differentSMILES == []:
            f.write(f'No duplicate InChIs with different SMILES in AID{AID}\n')
        else:
            f.write(f'Found duplicate InChIs with different SMILES in AID{AID}:\n')
            f.write('\n'.join(str(item) for item in sameInChI_differentSMILES))
            f.write("\n")
        f.write("===\n")

## 4.4. Same SMILES but with different CIDs or SMILES

In [ ]:
#reindex
for AID in AIDs:
    exec(f"AID{AID}_duplicates_SMILES.reset_index(drop=True, inplace=True)")

In [ ]:
with open(f'{data_folder}/before_finished/step_4/sameSMILES_different_others.txt', 'w') as f:
    for AID in AIDs: 
        sameSMILES_differentCID = []
        sameSMILES_differentInChI = []
        duplicates_SMILES = eval(f'AID{AID}_duplicates_SMILES')
        for i in range(len(duplicates_SMILES[smi_col])):
            for j in range(i+1, len(duplicates_SMILES[smi_col])):
                if duplicates_SMILES[smi_col][i] == duplicates_SMILES[smi_col][j]:
                    if duplicates_SMILES[cid_col][i] != duplicates_SMILES[cid_col][j]:
                        sameSMILES_differentCID.append((duplicates_SMILES[cid_col][i], duplicates_SMILES[cid_col][j]))
                    if duplicates_SMILES['InChI'][i] != duplicates_SMILES['InChI'][j]:
                        sameSMILES_differentInChI.append((duplicates_SMILES[cid_col][i], duplicates_SMILES[cid_col][j]))
        
        if sameSMILES_differentCID == []:
            f.write(f'No duplicate SMILES with different CIDs in AID{AID}\n')
        else:
            f.write(f'Found duplicate SMILES with different CIDs in AID{AID}:\n')
            f.write('\n'.join(str(item) for item in sameSMILES_differentCID))
            f.write("\n")
        
        if sameSMILES_differentInChI == []:
            f.write(f'No duplicate SMILES with different InChIs in AID{AID}\n')
        else:
            f.write(f'Found duplicate SMILES with different InChIs in AID{AID}:\n')
            f.write('\n'.join(str(item) for item in sameSMILES_differentInChI))
            f.write("\n")
        f.write("===\n")

## 4.5 Drop duplicates

When dropping duplicates, we will keep the first molecule in a pair or a group of duplicates. For example, here there are 12 duplicates (6 pairs) so we keep 6 of them.

In [ ]:
# Keep only the first duplicate in the dataframes:
for AID in AIDs: 
    exec(f"AID{AID}.drop_duplicates(subset=['InChI'], keep='first', inplace=True)")

    last_check = f"""
if len(AID{AID}[AID{AID}.duplicated(subset=['InChI'], keep=False)]) == 0:
    print('No more duplicate InChI in AID{AID}')
else:
    print('There are still duplicate InChI in AID{AID}')   
    """
    exec(last_check)

No more duplicate InChI in AID859
No more duplicate InChI in AID628
No more duplicate InChI in AID677
No more duplicate InChI in AID860


In [ ]:
for AID in AIDs: 
    last_check = f"""
if len(AID{AID}[AID{AID}.duplicated(subset=[smi_col], keep=False)]) == 0:
    print('No more duplicate SMILES in AID{AID}')
else:
    print('There are still duplicate SMILES in AID{AID}')   
    """
    exec(last_check)

In [ ]:
for AID in AIDs: 
    last_check = f"""
if len(AID{AID}[AID{AID}.duplicated(subset=[cid_col], keep=False)]) == 0:
    print('No more duplicate CID in AID{AID}')
else:
    print('There are still duplicate CID in AID{AID}')   
    """
    exec(last_check)

In [ ]:
# Save the dataframes to csv:
for AID in AIDs: 
    exec(f"AID{AID}.to_csv(r'{data_folder}/before_finished/step_4/AID{AID}.csv', index=False)")

# 5. Hierarchical Curation

For the hierarchical curation, there are some rules:

(1) All assays used should be on the same or close species/cell lines. Optimally, they should also be from the same project/laboratory.

(2) Primary actives (PrA) will have a large false-positive rate. Therefore, they should be tested in follow-up confirmatory screens (optimally dose-reponse).

(3) Actives could be promiscuous. Therefore, it is optimal to have counter-screens on different targets to test specificity.

(4) For some projects, compounds were tested in multiple rounds. Therefore, assays often have hierarchical relations. From a single primary screen (Pr), active compounds (Pr_A) could be tested in multiple rounds of confirmatory screens (Cf_1, Cf_2, ..., Cf_final) or counter screens (Ct_1, Ct_2, etc.). Actives from confirmatory screens (Cf_actives) have a higher possibility of being true active. If an active compound is tested active in counter screens (Cf_actives), it is likely to be a promiscuous compound and should not be included.

(4) It is important to know the relationship between assays. Active sets from downstream screens always have a lower false-positive rate than active sets from upstream screens due to better assay technologies on a smaller set of compounds. Therefore, final hits should be taken from the intersection of the very last confirmatory assays, without tested active in any counter-screen:
Final hits = [Cf_final1_actives ∩ Cf_final2_actives ∩ ...] \ [Ct_1_actives ∪ Ct_2_actives ∪ ...]

However, if the confirmatory assays are unrelated (tested on different set of compounds), then we might have to take the union of their active sets instead of the intersections as in this formula.

(5) The hierarchical relations should be inspected carefully to see if follow-up confirmatory screens include extra compounds (Ex) that were not tested in earlier screens or tested inactive in earlier screens. If exist, these compounds require manual inspection.

(6) Final inactives should be taken from primary inactives (Pr_inactives) (not inconclusive, unspecified, or probes), plus extra compounds that were tested inactive in conformatory screens (Ex_inactives), if justified.
Final inactives = Pr_inactives ∪ Ex_inactives

## 5.1. Classify groups of compounds in each assay by activities

In [ ]:
path = f'{data_folder}/before_finished/step_4'
keynumbers = [628, 677, 859, 860] # specify the keynumbers you want to import

for keynumber in keynumbers:
    filename = os.path.join(path, f'AID{keynumber}.csv')
    if os.path.exists(filename):
        df = pd.read_csv(filename, index_col=None, header=0)
        exec(f'AID{keynumber} = df')
        exec(f'AID{keynumber}_active = df[df["PUBCHEM_ACTIVITY_OUTCOME"]=="Active"]')
        exec(f'AID{keynumber}_inactive = df[df["PUBCHEM_ACTIVITY_OUTCOME"]=="Inactive"]')
        exec(f'AID{keynumber}_inconclusive = df[df["PUBCHEM_ACTIVITY_OUTCOME"]=="Inconclusive"]')
        exec(f'AID{keynumber}_unspecified = df[df["PUBCHEM_ACTIVITY_OUTCOME"]=="Unspecified"]')
        exec(f'AID{keynumber}_probe = df[df["PUBCHEM_ACTIVITY_OUTCOME"]=="Probe"]')

In [ ]:
#Create a df with first column the variables name, and the second column the number of rows:
df = pd.DataFrame(columns=['AID', 'Tested Compounds', 'Active', 'Inactive', 'Inconclusive', 'Unspecified', 'Probe'])
for keynumber in keynumbers:
    exec(f'df.loc[len(df)] = ["AID{keynumber}", len(AID{keynumber}), len(AID{keynumber}_active), len(AID{keynumber}_inactive), len(AID{keynumber}_inconclusive), len(AID{keynumber}_unspecified), len(AID{keynumber}_probe)]')
df

,AID,Tested Compounds,Active,Inactive,Inconclusive,Unspecified,Probe
0,AID628,63656,2179,61477,0,0,0
1,AID677,1665,723,942,0,0,0
2,AID859,591,231,360,0,0,0
3,AID860,719,272,447,0,0,0


## 5.2. Check the hierachical relations

In [ ]:
def check_is_in(downstream, upstream):
    downstream_in_upstream = downstream[downstream[cid_col].isin(upstream[cid_col])]
    downstream_notin_upstream = downstream[~downstream[cid_col].isin(upstream[cid_col])]
    return downstream_in_upstream, downstream_notin_upstream

### Flow: AID628 (Pr), AID677 (Cf_1), AID860 (Ct_1), and AID859 (Cf_2)

In [ ]:
a1, a2 = check_is_in(AID859, AID677_inactive)
a3, a4 = check_is_in(a1, AID859_active)
print(f'Among AID859, {len(a1)} were tested inactive in AID677. Among these, {len(a3)} became active')

b1, b2 = check_is_in(AID859, AID628_inactive)
b3, b4 = check_is_in(b1, AID859_active)
b5, b6 = check_is_in(AID859, AID628)
b7, b8 = check_is_in(b6, AID859_active)
print(f'Among AID859, {len(b1)} were tested inactive in AID628. Among these, {len(b3)} became active')
print(f'Among AID859, {len(b6)} were not tested in the AID628. Among these, {len(b7)} became active')

c1, c2 = check_is_in(AID859, AID860_active)
c3, c4 = check_is_in(c1, AID859_active)
print(f'Among AID859, {len(c1)} were tested active in the counter AID860. Among these, {len(c3)} became active')

Among AID859, 0 were tested inactive in AID677. Among these, 0 became active
Among AID859, 0 were tested inactive in AID628. Among these, 0 became active
Among AID859, 0 were not tested in the AID628. Among these, 0 became active
Among AID859, 219 were tested active in the counter AID860. Among these, 144 became active


Here, no extra compounds were found. The potential final hits therefore should be the confirmed actives from AID859, subtracted by compounds that were active in the counter screens. Inactives should be taken from the primary inactives.

## 5.3. Export the data

In [ ]:
potential_actives = AID859_active[~AID859_active[cid_col].isin(AID860_active[cid_col])]
potential_inactives = AID628_inactive

In [ ]:
if not os.path.exists(f'{data_folder}/before_finished/step_5'):
    os.makedirs(f'{data_folder}/before_finished/step_5')

In [ ]:
#export the potential hits and inactives to csv:
potential_actives.to_csv(f'{data_folder}/before_finished/step_5/potential_actives.csv', index=False)
potential_inactives.to_csv(f'{data_folder}/before_finished/step_5/potential_inactives.csv', index=False)

# 6. RDkit Parse Check

In [ ]:
potential_actives = pd.read_csv(f'{data_folder}/before_finished/step_5/potential_actives.csv', sep=',', header=0)
potential_inactives = pd.read_csv(f'{data_folder}/before_finished/step_5/potential_inactives.csv', sep=',', header=0)

In [ ]:
problems_actives, cannot_parse_actives = filters.rdkit_parse(potential_actives, smi_col, cid_col)
problems_inactives, cannot_parse_inactives = filters.rdkit_parse(potential_inactives, smi_col, cid_col)

No problems detected
No problems detected


In [ ]:
#Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_6'):
    os.makedirs(f'{data_folder}/before_finished/step_6')

with open(f'{data_folder}/before_finished/step_6/problem_list_actives.txt', 'w') as f:
    f.write("Problems:\n")
    for item in problems_actives:
        f.write("%s\n" % item)
    f.write("Cannot parse:\n")
    for item in cannot_parse_actives:
        f.write("%s\n" % item)

with open(f'{data_folder}/before_finished/step_6/problem_list_inactives.txt', 'w') as f:
    f.write("Problems:\n")
    for item in problems_inactives:
        f.write("%s\n" % item)
    f.write("Cannot parse:\n")
    for item in cannot_parse_inactives:
        f.write("%s\n" % item)

Our dataset returned no problem or non-parsable molecule.

# 7. Inorganics Filter

In [ ]:
# Import data
potential_actives = pd.read_csv(f'{data_folder}/before_finished/step_5/potential_actives.csv', sep=',', header=0)
potential_inactives = pd.read_csv(f'{data_folder}/before_finished/step_5/potential_inactives.csv', sep=',', header=0)

In [ ]:
inorganic_actives_cids, organic_actives_cids = filters.inorganic_filter(potential_actives, smi_col, cid_col, type='smiles')
print(f'In hits, there are {len(organic_actives_cids)} organic molecules and {len(inorganic_actives_cids)} inorganic molecules')

inorganic_inactives_cids, organic_inactives_cids = filters.inorganic_filter(potential_inactives, smi_col, cid_col, type='smiles')
print(f'In inactives, there are {len(organic_inactives_cids)} organic molecules and {len(inorganic_inactives_cids)} inorganic molecules')

In hits, there are 87 organic molecules and 0 inorganic molecules
In inactives, there are 61477 organic molecules and 0 inorganic molecules


In [ ]:
#Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_7'):
    os.makedirs(f'{data_folder}/before_finished/step_7')

with open(f'{data_folder}/before_finished/step_7/inorganic.txt', 'w') as f:
    f.write("Actives:\n")
    for item in inorganic_actives_cids:
        f.write("%s\n" % item)
    f.write("\n\nInactives:\n")
    for item in inorganic_inactives_cids:
        f.write("%s\n" % item)

# Drop inorganics: 
potential_actives = potential_actives[~potential_actives[cid_col].isin(inorganic_actives_cids)]
potential_inactives = potential_inactives[~potential_inactives[cid_col].isin(inorganic_inactives_cids)]

#save: 
potential_actives.to_csv(f'{data_folder}/before_finished/step_7/organic_actives.csv', index=False)
potential_inactives.to_csv(f'{data_folder}/before_finished/step_7/organic_inactives.csv', index=False)

print('Dropped inorganic and saved organic compounds into step_7 folder.')

# 8. Mixtures Handling

## 8.1. Quick check

In [ ]:
#import: 
organic_actives = pd.read_csv(f'{data_folder}/before_finished/step_7/organic_actives.csv', sep=',', header=0)
organic_inactives = pd.read_csv(f'{data_folder}/before_finished/step_7/organic_inactives.csv', sep=',', header=0)

In [ ]:
filters.quick_check_mixtures('hits', organic_actives[smi_col])
filters.quick_check_mixtures('inactives', organic_inactives[smi_col])

Total number of mixtures in hits is 21
Total number of mixtures in inactives is 1741


## 8.2. Handling mixture

In [ ]:
processed_actives, removed_actives, small_organic_actives, small_inorganic_actives, not_lipinski_actives, cleaned_actives = filters.process_smi_mixtures(organic_actives, smi_col, cid_col)
processed_inactives, removed_inactives, small_organic_inactives, small_inorganic_inactives, not_lipinski_inactives, cleaned_inactives = filters.process_smi_mixtures(organic_inactives, smi_col, cid_col)

Cannot decide between ['CC1=NC2=C(O1)C3=CC=CC=C3C=C2', 'C1=C(C=C(C(=C1[N+](=O)[O-])O)[N+](=O)[O-])[N+](=O)[O-]'] for CID 3241713
Big organic molecule for CID 3244813 does not pass Lipinski's rule of five
Cannot decide between ['CN(C)C1=NC2=CC=CC=C2C(=C1)N', 'C1=C(NC(=O)NC1=O)C(=O)O'] for CID 646688
Cannot decide between ['C1C2(CN3CN1CN(C2)C3)N', 'C1=CC(=CN=C1)C(=O)O'] for CID 648270
Cannot decide between ['CN(C)C1=NC2=C(CCC2)C(=C1)N', 'C1=C(C=NC=C1O)C(=O)O'] for CID 652177
Cannot decide between ['CC1=NC2=C(S1)C3=CC=CC=C3C=C2', 'C1=C(C=C(C(=C1[N+](=O)[O-])O)[N+](=O)[O-])[N+](=O)[O-]'] for CID 654127
Cannot decide between ['C1CC1C(C2CC2)NC3=NCCO3', 'C(=C/C(=O)O)\\C(=O)O'] for CID 5388964
Cannot decide between ['CN1C(=O)C2=C(N=C(N2)Cl)N(C1=O)C(=O)[O-]', 'CN(C)CCOC(C1=CC=CC=C1)C2=CC=CC=C2'] for CID 657227
Cannot decide between ['CC[N+](C)(C)CC1=CC=CC=C1Br', 'CC1=CC=C(C=C1)S(=O)(=O)[O-]'] for CID 6100
Cannot decide between ['C1CN(CCN1CCOCCO)C(C2=CC=CC=C2)C3=CC=C(C=C3)Cl', 'C1=CC=C2C(=C1)C=C

In [ ]:
# Create a new step folder
if not os.path.exists(f'{data_folder}/before_finished/step_8'):
    os.makedirs(f'{data_folder}/before_finished/step_8')

#Generate df with the smiles column in the cleaned_actives or cleaned_inactives dictionary:
cleaned_actives_df = pd.DataFrame(list(cleaned_actives.values()), columns=[smi_col])
cleaned_inactives_df = pd.DataFrame(list(cleaned_inactives.values()), columns=[smi_col])

#Export the cleaned hits and inactives to csv:
cleaned_actives_df.to_csv(f'{data_folder}/before_finished/step_8/cleaned_mixtures_actives.csv', index=False, header=False)
cleaned_inactives_df.to_csv(f'{data_folder}/before_finished/step_8/cleaned_mixtures_inactives.csv', index=False, header=False)

In [ ]:
processed_hits_df = filters.process_mixture_df('actives_M1_antagonist', organic_actives, processed_actives, removed_actives, small_organic_actives, small_inorganic_actives, smi_col, cid_col)
processed_inactives_df = filters.process_mixture_df('inactives_M1_antagonist', organic_inactives, processed_inactives, removed_inactives, small_organic_inactives, small_inorganic_inactives, smi_col, cid_col)

3241713 has been removed from inactives_M1_antagonist because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.
3244813 has been removed from inactives_M1_antagonist because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.
646688 has been removed from inactives_M1_antagonist because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.
648270 has been removed from inactives_M1_antagonist because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.
652177 has been removed from inactives_M1_antagonist because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.
654127 has been removed from inactives_M1_antagonist because it is a mixture with less than 5 atoms differen

In [ ]:
with open(f'{data_folder}/before_finished/step_8/mixture.txt', 'w') as f:
    f.write(f"""
Hits before processing: {len(organic_actives)}
Hits before processing: {len(organic_actives)}
Hits after processing: {len(processed_hits_df)}
Mixtures detected: {len(removed_actives)}
Mixtures with small inorganic molecules: {len(small_inorganic_actives)}
Mixtures with big organic molecules passing Lipinski: {len(small_organic_actives)}
Mixtures with big organic molecules not passing Lipinski: {len(not_lipinski_actives)}

Inactives before processing: {len(organic_inactives)}
Inactives after processing: {len(processed_inactives_df)}
Mixtures detected: {len(removed_inactives)}
Mixtures with small inorganic molecules: {len(small_inorganic_inactives)}
Mixtures with big organic molecules passing Lipinski: {len(small_organic_inactives)}
Mixtures with big organic molecules not passing Lipinski: {len(not_lipinski_inactives)}
""")

# Save the processed dataframes to csv
processed_hits_df.to_csv(f'{data_folder}/before_finished/step_8/post8_actives.csv', index=False)
processed_inactives_df.to_csv(f'{data_folder}/before_finished/step_8/post8_inactives.csv', index=False)

print('Dataframes saved successfully')

Dataframes saved successfully


# 9. Neutralize & 10. Aromatize Molecules

In [ ]:
# Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_9_10'):
    os.makedirs(f'{data_folder}/before_finished/step_9_10')

#import:
pre9_actives = pd.read_csv(f'{data_folder}/before_finished/step_8/post8_actives.csv', sep=',', header=0)
pre9_inactives = pd.read_csv(f'{data_folder}/before_finished/step_8/post8_inactives.csv', sep=',', header=0)

In [ ]:
updated_smi = []

#Update dataset with neutralized, aromatic SMILES
for smi in pre9_actives[smi_col]: 
    mol = Chem.MolFromSmiles(smi)
    mol_neu = utils.neutralize_atoms(mol)
    smi_arom = utils.aromatize_smile(mol_neu)
    updated_smi.append(smi_arom)
    
#update the smiles in this df
pre9_actives[smi_col] = updated_smi

In [ ]:
updated_smi = []

#Update dataset with neutralized, aromatic SMILES
for smi in pre9_inactives[smi_col]: 
    mol = Chem.MolFromSmiles(smi)
    mol_neu = utils.neutralize_atoms(mol)
    smi_arom = utils.aromatize_smile(mol_neu)
    updated_smi.append(smi_arom)

#update the smiles in this df
pre9_inactives[smi_col] = updated_smi

In [ ]:
#Save
pre9_actives.to_csv(f'{data_folder}/before_finished/step_9_10/post10_actives.csv', index=False)
pre9_inactives.to_csv(f'{data_folder}/before_finished/step_9_10/post10_inactives.csv', index=False)

# Post 9+10: Update InChI


Is it important to now update InChI in our datasets, for 2 reasons:

(1) Some mixture compounds have been modified (removal of small inorganic or organic molecules) in SMILES representation but not InChIs.

(2) The SMILES representations have been neutralized and aromatized, but not InChIs.

In [ ]:
#export the smiles columns to txt
pre9_actives[smi_col].to_csv(f'{data_folder}/before_finished/step_9_10/smiles_actives.txt', index=False, header=False)
pre9_inactives[smi_col].to_csv(f'{data_folder}/before_finished/step_9_10/smiles_inactives.txt', index=False, header=False)

In [ ]:
"""
Submit the smiles files to PubChem Identifier Exchange Service: 
    Input IDs: "SMILES"
    Operator type: "same CID" 
    Output IDs: "InChI"
    Output method: "Two column file showing each input output-correspondence"
    Compression: "No compression"
InChI list should be saved into "step_9_10" folder, named as "inchi_actives.txt" and "inchi_inactives" 
"""
#Import the converted InChIs
cleaned_inchi_hits = pd.read_csv(f'{data_folder}/before_finished/step_9_10/inchi_actives.txt', sep='\t', header=None)
cleaned_inchi_inactives = pd.read_csv(f'{data_folder}/before_finished/step_9_10/inchi_inactives.txt', sep='\t', header=None)

#a dictionary of smiles and corresponding inchi in cleaned_inchi_hits
hits_smi_inchi_dict = dict(zip(cleaned_inchi_hits[0], cleaned_inchi_hits[1]))
inactives_smi_inchi_dict = dict(zip(cleaned_inchi_inactives[0], cleaned_inchi_inactives[1]))
                             
#update the pre9_hits by matching the smiles with keys and replace inchi with values:
pre9_actives['InChI'] = pre9_actives[smi_col].map(hits_smi_inchi_dict) 
pre9_inactives['InChI'] = pre9_inactives[smi_col].map(inactives_smi_inchi_dict)

In [ ]:
#export: 
pre9_actives.to_csv(f'{data_folder}/before_finished/step_9_10/post10_actives.csv', index=False)
pre9_inactives.to_csv(f'{data_folder}/before_finished/step_9_10/post10_inactives.csv', index=False)

# 11. PAIN Filters

## 11.1. Frequency of Hits (FoH) Filter

Frequency of Hits is a complex concept that requires a merticulous approach. In general, the rule is if a compound was tested active in multiple assays, it is likely to be a promiscuous compound.
1. For each compounds, retrieve the information on its tested assays
2. For each of the assay tested, retrieve the sequence of the protein target.
3. Given all sequence of the protein tested, do a multiple sequence alignment to find the percentage Percent Identity (similarty) between these proteins. If an assay has high percentage to other targets, then these assays contribute less to promiscuousity of the compound.
4. Use the percentage identity as a weight:
w = 1 - %SI/100
Calculate the frequency of hits for each compound:
FoH = wACC/TAC
wACC is the weighed total number of assay tested where the compounds were identified acitives. TAC is the total number of assays tested.

In [ ]:
pre11_actives = pd.read_csv(f'{data_folder}/before_finished/step_9_10/post10_actives.csv', sep=',', header=0)
pre11_inactives = pd.read_csv(f'{data_folder}/before_finished/step_9_10/post10_inactives.csv', sep=',', header=0)

# Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_11/11_1'):
    os.makedirs(f'{data_folder}/before_finished/step_11/11_1')

### 11.1.1. PubChem testing information for each compound

This part illustrates how to retrieve the information of how each compound was tested from the PubChem database. Bulk data retrieval from the ftp server is used to get the information of every bioassay in PubChem:

In [ ]:
url = 'https://ftp.ncbi.nlm.nih.gov/pubchem/Bioassay/Extras/bioassays.tsv.gz' #this FTP file records the summary data of all available AIDs in PubChem

local_save_dir = 'S:\coding\WelQrate\pubchem_sum'
local_save_path = os.path.join(local_save_dir, 'bioassays.tsv.gz')

if not os.path.exists(local_save_dir):
    os.makedirs(local_save_dir)
r = requests.get(url, stream=True)

with open(local_save_path, 'wb') as f:
    for chunk in r.iter_content(chunk_size=8192):
        f.write(chunk)
print('Downloaded to %s' % local_save_path)

path = 'pubchem_sum/bioassays.tsv.gz'

# Read the TSV file
all_bioassay = pd.read_csv(path, delimiter='\t')

In [ ]:
all_bioassay.head()

,AID,BioAssay Name,Deposit Date,Modify Date,Source Name,Source ID,Substance Type,Outcome Type,Project Category,BioAssay Group,BioAssay Types,Protein Accessions,UniProts IDs,Gene IDs,Target TaxIDs,Taxonomy IDs,Number of Tested SIDs,Number of Active SIDs,Number of Tested CIDs,Number of Active CIDs
0,1,NCI human tumor cell line growth inhibition as...,20040815,20231014,DTP/NCI,NCI human tumor cell line growth inhibition as...,small-molecule,Confirmatory,Other,NCI-60_DOSERESP,NaN,NaN,NaN,NaN,NaN,NaN,55004,3280,52994,3057
1,3,NCI human tumor cell line growth inhibition as...,20040815,20231014,DTP/NCI,NCI human tumor cell line growth inhibition as...,small-molecule,Confirmatory,Other,NCI-60_DOSERESP,NaN,NaN,NaN,NaN,NaN,NaN,51204,2596,49337,2449
2,5,NCI human tumor cell line growth inhibition as...,20040815,20231014,DTP/NCI,NCI human tumor cell line growth inhibition as...,small-molecule,Confirmatory,Other,NCI-60_DOSERESP,NaN,NaN,NaN,NaN,NaN,NaN,53832,2486,51804,2301
3,7,NCI human tumor cell line growth inhibition as...,20040815,20231014,DTP/NCI,NCI human tumor cell line growth inhibition as...,small-molecule,Confirmatory,Other,NCI-60_DOSERESP,NaN,NaN,NaN,NaN,NaN,NaN,53828,4275,51804,4041
4,9,NCI human tumor cell line growth inhibition as...,20040815,20231014,DTP/NCI,NCI human tumor cell line growth inhibition as...,small-molecule,Confirmatory,Other,NCI-60_DOSERESP,NaN,NaN,NaN,NaN,NaN,NaN,53744,3125,51771,2948


### 11.1.2 Retrieving protein sequences for assays tested:

Then, the testing information for each compound is retrieved from the PugREST API

In [ ]:
# Cache to store the number of compounds tested per AID to avoid redundant call. 
num_compounds_tested_cache = {}

def get_num_compounds_tested(aid, all_bioassay=all_bioassay):
    """
    This function retrieves the information of how many compounds were tested in a given assay (by AID).
    """
    if aid in num_compounds_tested_cache:
        return num_compounds_tested_cache[aid]
    else: 
        #return the 'Number of Tested CIDs' column value at the row where the 'AID' column is equal to aid in the all_bioassay dataframe
        num_compounds_tested = all_bioassay[all_bioassay['AID'] == aid]['Number of Tested CIDs'].values[0]
    return num_compounds_tested

def get_assay_data(cid):
    """
    Return a dictionary of all targets that a given compound (by CID) was tested on in PubChem 
    and the activity values of the compound. 
    """
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/assaysummary/JSON" #PUG-REST compound summary by CID
    response = requests.get(url)
    data = response.json()

    target_activity = {}

    if 'Table' in data and 'Row' in data['Table']:
        for row in data['Table']['Row']:
            cells = row['Cell']
            aid = int(cells[0])  # Extracting the AID from the first cell

            # Proceed only if the assay is a screening assay
            if cells[10] == 'Screening':

                # Proceed only if more than 10,000 compounds were tested
                num_compounds_tested = get_num_compounds_tested(aid)
                if num_compounds_tested > 10000:
                    target_gi = cells[5] # Retrieve the protein target's GI
                    activity_outcome = cells[4].lower()

                    if target_gi not in target_activity:
                        target_activity[target_gi] = activity_outcome == 'active'
                    elif activity_outcome == 'active':
                        target_activity[target_gi] = True # If a compound was tested multiple times on the same protein, priotize "active" outcome.
            
            else:
                continue

    return (cid, target_activity)

In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

cids_list = pre11_actives[cid_col].tolist()

def execute_with_multiprocessing(cids_list):
    """
    For a given list of CIDs, return a dictionary of dictionaries 
    of protein targets these compounds were tested on and the activity outcomes
    Input: 
        [list of CIDs]
    Output: 
        Dictionary of testing information for all CIDs, such as:
        {CID1:{target1:activity1, target3:activity3, ...},{CID2:{target2:activity2, target4:activity4, ...}, ...}}
    """
    results_dict = {}
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Prepare futures for all CIDs
        futures = [executor.submit(get_assay_data, cid) for cid in cids_list]
        
        # Process futures as they complete
        for future in tqdm(as_completed(futures), total=len(cids_list), desc="Processing CIDs"):
            try:
                cid, target_activity = future.result()
                results_dict[cid] = target_activity
            except Exception as e:
                print(f"Error processing CID: {e}")
    return results_dict

results_dict = execute_with_multiprocessing(cids_list)

Processing CIDs: 100%|██████████| 87/87 [00:27<00:00,  3.19it/s]


In [ ]:
#export results_dict
with open(f'{data_folder}/before_finished/step_11/11_1/results_dict.json', 'w') as f:
    json.dump(results_dict, f)

In [ ]:
#import results_dict:
with open(f'{data_folder}/before_finished/step_11/11_1/results_dict.json', 'r') as f:
    results_dict = json.load(f)

#get the list of all keys of the values in the dictionary:
protein_ids = []
for value in results_dict.values():
    protein_ids.extend(value.keys())

#clean the list
protein_ids = list(set(protein_ids))
protein_ids = [id for id in protein_ids if id != '']

print(protein_ids)
print(f'Require multiple sequencing alignment for {len(protein_ids)} proteins.')

['90111653', '291463269', '13699818', '30219', '47496637', '2853980', '4505445', '998701', '15646160', '124486680', '4503155', '118341367', '2358024', '21264324', '38349113', '116907', '23505220', '119579215', '134244587', '119580345', '4503385', '115529463', '10835013', '70832125', '46577642', '74315350', '14790033', '63102437', '5174513', '119603173', '2501205', '28373962', '262118306', '190938', '6016094', '4885057', '56417702', '21955158', '40254439', '111305821', '6274552', '7657550', '15645703', '32425330', '223468676', '4507793', '21361095', '115347926', '730163', '16878311', '156416009', '4826706', '32307126', '231632', '257380', '56202836', '155969707', '6912644', '23893623', '12830367', '171229', '47132585', '89348172', '56790945', '62868213', '78486550', '5454102', '216548193', '13325293', '4758204', '73586699', '73745819', '90652859', '52426748', '4502331', '166202459', '124487323', '89993689', '67463988', '15929025', '6680530', '4758878', '341916350', '13236497', '499328',

Now we retrieve all the FASTA sequences of proteins tested for all of our compounds with Biopython API to Entrez of NCBI. The FASTA sequence is saved as "sequences.fasta"

In [ ]:
# Always tell NCBI who you are
Entrez.email = "hdong26@amherst.edu"

# The filename where you want to save the sequences
output_filename = f'{data_folder}/before_finished/step_11/11_1/sequences.fasta'

# Open a file to write the sequences
with open(output_filename, "w") as output_file:
    for id in protein_ids:
        try:
            # Fetch the sequence from NCBI
            handle = Entrez.efetch(db="protein", id=id, rettype="fasta", retmode="text")
            sequence_data = handle.read()
            handle.close()

            # Write the sequence data to the file
            output_file.write(sequence_data)
        except Exception as e:
            print(f"An error occurred while fetching {id}: {e}")

From the FASTA sequence, we also need to retrieve the list of protein names, since these are different from the protein GIs

In [ ]:
def extract_protein_names(file_path):
    protein_names = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('>'):
                # Split the line at spaces and take the first item
                parts = line.split(' ')
                protein_name = parts[0]
                # Remove the leading '>' character
                protein_name = protein_name[1:]
                protein_names.append(protein_name)
    return protein_names

file_path = f'{data_folder}/before_finished/step_11/11_1/sequences.fasta'
protein_names = extract_protein_names(file_path)

# Create a dictionary to map protein IDs to protein names by index 
protein_id_to_name = {protein_ids[i]: protein_names[i] for i in range(len(protein_ids))}

### 11.1.3 Percent Sequence Identity by Multiple Sequence Alignment

The sequences.fasta file is submitted to https://www.ebi.ac.uk/jdispatcher/msa/clustalo for multiple sequencing alignment. The resulted table of percent sequence identity matrix is saved and imported for the calculation of FoH

In [ ]:
"""
Submit sequences.fasta to https://www.ebi.ac.uk/jdispatcher/msa/clustalo
    Input sequence type: Protein
    Output format: ClustalW with character counts
Download the resulted Percent Identity Matrix file file and save as "percent_identity_matrix.txt"
"""

#import the identity matrix:
protein_si = pd.read_csv(
    f'{data_folder}/before_finished/step_11/11_1/percent_identity_matrix.txt',
    delimiter='\s+',
    header=None,
    skiprows=6 
)

In [ ]:
#remove the first column:
protein_si = protein_si.drop(protein_si.columns[0], axis=1)

name = protein_si[1].tolist()
name = ['protein name'] + name
protein_si.columns = name

In [ ]:
protein_si

,protein name,NP_063937.2,NP_002084.2,NP_000302.1,NP_004447.2,NP_057856.1,NP_001123617.1,NP_001357589.1,NP_492143.1,NP_036543.4,...,NP_001136020.1,NP_001167549.1,EAW58811.1,NP_217253.1,EAW82767.1,NP_066921.2,AAA62473.1,NP_004944.3,NP_004283.2,NP_004095.4
0,NP_063937.2,100.00,76.85,15.69,13.91,15.68,8.22,14.77,10.42,14.56,...,6.94,8.09,6.87,10.00,12.07,7.69,7.95,5.45,7.89,12.64
1,NP_002084.2,76.85,100.00,17.65,14.02,15.73,9.72,11.88,10.53,12.42,...,6.76,9.84,8.55,11.11,12.77,8.70,12.50,9.66,5.88,12.05
2,NP_000302.1,15.69,17.65,100.00,11.11,16.82,17.20,5.98,12.15,11.86,...,5.97,5.06,5.06,NaN,10.53,14.39,8.79,8.11,16.67,13.33
3,NP_004447.2,13.91,14.02,11.11,100.00,15.94,12.43,10.51,13.64,11.64,...,7.01,10.91,10.80,8.33,13.16,10.00,9.21,8.81,8.33,11.81
4,NP_057856.1,15.68,15.73,16.82,15.94,100.00,11.92,9.65,10.75,11.50,...,13.68,12.62,10.80,12.00,16.16,9.52,13.41,8.50,8.33,12.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,NP_066921.2,7.69,8.70,14.39,10.00,9.52,10.78,12.85,14.43,10.61,...,18.18,19.10,17.38,20.27,19.38,100.00,16.21,16.93,15.37,19.26
417,AAA62473.1,7.95,12.50,8.79,9.21,13.41,9.85,9.09,4.63,9.52,...,9.04,10.83,12.28,12.62,16.04,16.21,100.00,16.28,14.60,20.41
418,NP_004944.3,5.45,9.66,8.11,8.81,8.50,13.70,9.68,9.43,11.03,...,13.10,17.65,17.04,14.77,12.78,16.93,16.28,100.00,17.60,21.53
419,NP_004283.2,7.89,5.88,16.67,8.33,8.33,8.77,9.20,9.38,3.80,...,14.75,16.67,15.56,11.46,15.98,15.37,14.60,17.60,100.00,25.11


### 11.1.4 Calculation of FoHs:

Until now, we have a dictionary of (cid: assays tested); (assay_tested:protein name), and percentage identity matrix with first columns as protein names.
For each compound, we retrieve the list of all protein names tested on that compounds by matching between the two first dictionary. From this list, we retrieve the corresponding matrix of percentages identitiy of these proteins corresponding to these compounds and calculate the FoH

In [ ]:
protein_si_dict = {}
for name in protein_si['protein name']: 
    for other_name in protein_si['protein name']: 
        if other_name != name: 
            protein_si_dict[(name, other_name)] = protein_si.loc[protein_si['protein name'] == name, other_name].values[0]

In [ ]:
foh_dict = {}

for cid, targets in tqdm.tqdm(results_dict.items()):
    active_weight_list = []
    total_weight_list = []

    for target_id, result in targets.items():
        if target_id == '':
            continue

        protein_name = protein_id_to_name[target_id]
        max_weight = 0

        for other_id, other_result in targets.items():
            if other_id != target_id and other_id != '':
                other_protein_name = protein_id_to_name[other_id]
                value = protein_si_dict[(protein_name, other_protein_name)]
                max_weight = max(max_weight, value)

        target_weight = 1 - max_weight / 100

        if result:
            active_weight_list.append(target_weight)
        total_weight_list.append(target_weight)

    if total_weight_list:
        foh_score = sum(active_weight_list) / sum(total_weight_list)
        foh_dict[cid] = foh_score
    else: 
        foh_dict[cid] = 0

100%|██████████| 87/87 [00:11<00:00,  7.79it/s]


In [ ]:
#export foh_dict
with open(f'{data_folder}/before_finished/step_11/11_1/foh_dict.json', 'w') as f:
    json.dump(foh_dict, f)

For compounds with FoH larger than 0.26, we remove them

In [ ]:
to_drop = []
for cid, foh_score in foh_dict.items():
    if foh_score > 0.26: 
        to_drop.append(cid)

post_FoH_actives = pre11_actives[~pre11_actives[cid_col].isin(to_drop)]
print(f'Dropped {(len(to_drop))} compounds with FoH larger than 0.26')

Dropped 0 compounds with FoH larger than 0.26


In [ ]:
post_FoH_actives.to_csv(f'{data_folder}/before_finished/step_11/11_1/post_FoH_actives.csv', index=False)
pre11_inactives.to_csv(f'{data_folder}/before_finished/step_11/11_1/post_FoH_inactives.csv', index=False)

## 11.2 Autofluoresence Filter

In [ ]:
post_FoH_actives = pd.read_csv(f'{data_folder}/before_finished/step_11/11_1/post_FoH_actives.csv', sep=',', header=0)
post_FoH_inactives = pd.read_csv(f'{data_folder}/before_finished/step_11/11_1/post_FoH_inactives.csv', sep=',', header=0)

When finding false positive due to autofluorescence and luceferase inhibition, it is important to check if the particular assays use one of these technologies. Here, all three assays (AID626, AID1488, and AID1741) use fluorescence technologies, so it is optimal to remove compounds that are active in AIDs: 587, 588, 590, 591, 592, 593, 594

In [ ]:
autofluorescence_cids = filters.load_autofluorescence_cids(data_folder)

In [ ]:
to_drop_actives = []
for cid in post_FoH_actives: 
    if cid in autofluorescence_cids:
        to_drop.append(cid)
post_autofluorescence_actives = post_FoH_actives[~post_FoH_actives['PUBCHEM_CID'].isin(to_drop_actives)]
print(f'Dropped {(len(to_drop_actives))} autofluorescence compounds')

Dropped 0 autofluorescence compounds


In [ ]:
post_autofluorescence_actives

,PUBCHEM_CID,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME,InChI,Mol removed from mixture,Small inorganic molecule,Small organic molecule
0,1263872,Cc1cc(N2CCN(c3nc(N4CCOCC4)nc4ccccc34)CC2)c2ccc...,Active,InChI=1S/C26H28N6O/c1-19-18-24(20-6-2-4-8-22(2...,NaN,NaN,NaN
1,655606,Cc1ccc2nc(NC3=NCN(CCN4CCOCC4)CN3)nc(C)c2c1,Active,InChI=1S/C19H27N7O/c1-14-3-4-17-16(11-14)15(2)...,NaN,NaN,NaN
2,3236651,C=CCSc1nnc2c(n1)OC(c1cccnc1)N(C(C)=O)c1ccccc1-2,Active,InChI=1S/C20H17N5O2S/c1-3-11-28-20-22-18-17(23...,NaN,NaN,NaN
3,6603423,Brc1cccc(-c2cnc3n2CCCCC3)c1,Active,InChI=1S/C14H15BrN2/c15-12-6-4-5-11(9-12)13-10...,Cl,Cl,NaN
4,2998899,Cc1ccccc1C(OCC(O)CN1CCCCC1CCO)c1ccccc1,Active,InChI=1S/C24H33NO3/c1-19-9-5-6-13-23(19)24(20-...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
82,3238898,CCOC(=O)N1CCN(C(=O)c2ccc3c(c2)sc2nc(-c4ccccc4)...,Active,InChI=1S/C23H22N4O3S/c1-2-30-23(29)26-12-10-25...,NaN,NaN,NaN
83,2999354,CN(C(=O)COC(=O)c1nc(-c2ccccc2)n(-c2ccccc2)n1)C...,Active,"InChI=1S/C22H22N4O5S/c1-25(18-12-13-32(29,30)1...",NaN,NaN,NaN
84,663466,CN1CCN(CC#CCn2c3ccccc3c3ccccc32)CC1,Active,InChI=1S/C21H23N3/c1-22-14-16-23(17-15-22)12-6...,NaN,NaN,NaN
85,2967872,CC(=O)c1ccccc1OCC(O)CN1CCN(C(c2ccccc2)c2ccccc2...,Active,InChI=1S/C28H32N2O3/c1-22(31)26-14-8-9-15-27(2...,NaN,NaN,NaN


In [ ]:
if not os.path.exists(f'{data_folder}/before_finished/step_11/11_2'):
    os.makedirs(f'{data_folder}/before_finished/step_11/11_2')

#save: 
post_autofluorescence_actives.to_csv(f'{data_folder}/before_finished/step_11/11_2/post_autofluorescence_actives.csv', index=False)
pre11_inactives.to_csv(f'{data_folder}/before_finished/step_11/11_2/post_autofluorescence_inactives.csv', index=False)

## 11.3 RDKit PAIN filter

In [ ]:
post_autofluorescence_actives = pd.read_csv(f'{data_folder}/before_finished/step_11/11_2/post_autofluorescence_actives.csv', sep=',', header=0)
post_autofluorescence_inactives = pd.read_csv(f'{data_folder}/before_finished/step_11/11_2/post_autofluorescence_inactives.csv', sep=',', header=0)

In [ ]:
pains_actives = filters.detect_pains(post_autofluorescence_actives, smi_col, cid_col)

1 pains detected
2 pains detected
3 pains detected
4 pains detected
5 pains detected
6 pains detected
7 pains detected


In [ ]:
post_pains_actives = post_autofluorescence_actives[~post_autofluorescence_actives['PUBCHEM_CID'].isin(pains_actives)]

In [ ]:
if not os.path.exists(f'{data_folder}/before_finished/step_11/11_3'):
    os.makedirs(f'{data_folder}/before_finished/step_11/11_3')

post_pains_actives.to_csv(f'{data_folder}/before_finished/step_11/11_3/post_pains_actives.csv', index=False)
post_autofluorescence_inactives.to_csv(f'{data_folder}/before_finished/step_11/11_3/post_pains_inactives.csv', index=False)

# 12. Drug-likeness Filter

In [ ]:
pre12_actives = pd.read_csv(f'{data_folder}/before_finished/step_11/11_3/post_pains_actives.csv', sep=',', header=0)
pre12_inactives = pd.read_csv(f'{data_folder}/before_finished/step_11/11_3/post_pains_inactives.csv', sep=',', header=0)

In [ ]:
not_drug_actives = filters.drug_likeness_filter_multiprocessing(pre12_actives, smi_col, cid_col)

Processing SMILES: 100%|██████████| 80/80 [00:00<?, ?it/s]


In [ ]:
not_drug_inactives = filters.drug_likeness_filter_multiprocessing(pre12_inactives, smi_col, cid_col)

Processing SMILES: 100%|██████████| 61464/61464 [00:00<00:00, 189179.37it/s]


In [ ]:
post12_actives = pre12_actives[~pre12_actives['PUBCHEM_CID'].isin(not_drug_actives)]
post12_inactives = pre12_inactives[~pre12_inactives['PUBCHEM_CID'].isin(not_drug_inactives)]

print(f'Dropped {(len(not_drug_actives))} hit compounds that do not pass the drug likeness filter')
print(f'Dropped {(len(not_drug_inactives))} inactive compounds that do not pass the drug likeness filter')

Dropped 2 hit compounds that do not pass the drug likeness filter
Dropped 1171 inactive compounds that do not pass the drug likeness filter


In [ ]:
if not os.path.exists(f'{data_folder}/before_finished/step_12'):
    os.makedirs(f'{data_folder}/before_finished/step_12')

#Export not_drug_actives and inactives:
with open(f'{data_folder}/before_finished/step_12/not_drug_actives.json', 'w') as f:
    json.dump(not_drug_actives, f)
with open(f'{data_folder}/before_finished/step_12/not_drug_inactives.json', 'w') as f:
    json.dump(not_drug_inactives, f)

In [ ]:
# save: 
post12_actives.to_csv(f'{data_folder}/before_finished/step_12/post12_actives.csv', index=False)
post12_inactives.to_csv(f'{data_folder}/before_finished/step_12/post12_inactives.csv', index=False)

# 13. ChemBL Curation Pipeline

Besides PubChem, the ChEMBL database is one of several public databases containing bioactivity data on small molecule compounds curated from various sources. Incoming compounds are typically not standardized according to consistent rules. To maintain the quality of the final database and to facilitate the comparison and integration of data on the same compound from different sources, it is essential to appropriately standardize the chemical structures in the database. This chemical curation pipeline has been developed by Bento, A.P, et al., using the open-source toolkit RDKit, including a Checker module that tests the validity of chemical structures and flags any serious errors. For ChEMBL, a penalty score of 7 is considered a fatal error, and the molfile is not loaded into the database.

In our dataset, we passed all compounds through this Checker module to evaluate the validity of the included chemical structures. For compounds that returned a penalty score of 7, we will check them manually.

Reference:
Bento, A.P., Hersey, A., Félix, E. et al. An open source chemical structure curation pipeline using RDKit. J Cheminform 12, 51 (2020). https://doi.org/10.1186/s13321-020-00456-1

In [ ]:
pre13_actives = pd.read_csv(f'{data_folder}/before_finished/step_12/post12_actives.csv', sep=',', header=0)
pre13_inactives = pd.read_csv(f'{data_folder}/before_finished/step_12/post12_inactives.csv', sep=',', header=0)

In [ ]:
# Apply the ChemBL Curation Pipeline Checker module:
score_actives = utils.checker_multiprocessing(pre13_actives, smi_col, cid_col)
score_inactives = utils.checker_multiprocessing(pre13_inactives, smi_col, cid_col)

Processing SMILES: 100%|██████████| 60293/60293 [00:30<00:00, 1989.84it/s] 


In [ ]:
#print all unique values in the dictionary:
print(set(score_actives.values()))
print(set(score_inactives.values()))

{0, 2}
{0, 2, 5, 6}


In [ ]:
# Create a new step folder
if not os.path.exists(f'{data_folder}/before_finished/step_13'):
    os.makedirs(f'{data_folder}/before_finished/step_13')

# save the scores:
with open(f'{data_folder}/before_finished/step_13/score_actives.json', 'w') as f:
    json.dump(score_actives, f)
with open(f'{data_folder}/before_finished/step_13/score_inactives.json', 'w') as f:
    json.dump(score_inactives, f)

In [ ]:
#drop all compounds with a penalty score of 7:
to_drop_actives = []
to_drop_inactives = []
for cid, penalty_score in score_actives.items():
    if penalty_score == 7:
        to_drop_actives.append(cid)
for cid, penalty_score in score_inactives.items():
    if penalty_score == 7:
        to_drop_inactives.append(cid)

post13_actives = pre13_actives[~pre13_actives[cid_col].isin(to_drop_actives)]
post13_inactives = pre13_inactives[~pre13_inactives[cid_col].isin(to_drop_inactives)]

In [ ]:
#save final_hits and inactives:
post13_actives.to_csv(f'{data_folder}/before_finished/step_13/post13_actives.csv', index=False)
post13_inactives.to_csv(f'{data_folder}/before_finished/step_13/post13_inactives.csv', index=False)

# 14. Final handling of chemical representation

Any kind of molecular processing should come with a special attention to post-processing adjustment. This is because during molecular processing, changes in data representation and format could lead to inconsistency or redundancy in the datasets. Below are the two adjustments performed on our datasets. 

(1) Some of the InChIs in our datasets should be updated. This is because some of the InChI will be missing, since the PubChem Identifier Exchange service might not able to find the corresponding InChI for the aromatized, neutralized SMILES.

(2) Presence of some additional duplicates resulted from molecular processing: While handling mixtures, there might be some mixtures whose component molecules are identical. For example, mixtures of organic and inorganic molecules (molX-ionA and molX-ionB) after removed the ions (ionA and ionB) will result in duplicates (molX). Moreover, since the original mixtures are different, their activities could be different. Therefore, we also need to check their activities while handling these duplicates.
- If all duplicates share the same results (active/inactive), we keep one of them, since it is likely that the organic molecule kept contributed more significantly to the activity of the mixture. 
- If duplicates of the same molecules returned different activity, it is safer to remove both of them.

In [ ]:
#import: 
pre14_actives = pd.read_csv(f'{data_folder}/before_finished/step_13/post13_actives.csv', sep=',', header=0)
pre14_inactives = pd.read_csv(f'{data_folder}/before_finished/step_13/post13_inactives.csv', sep=',', header=0)

## 14.1 Update InChI

In [ ]:
def smi_to_inchi(smi):
    mol = Chem.MolFromSmiles(smi)
    inchi = Chem.inchi.MolToInchi(mol)
    return inchi

In [ ]:
count = 0 
for index, row in pre14_actives.iterrows():
    if row['InChI'] != row['InChI']:
        pre14_actives.at[index, 'InChI'] = smi_to_inchi(row[smi_col])
        count += 1
print(f'Updated {count} InChI values in pre14_actives')

count = 0
for index, row in pre14_inactives.iterrows():
    if row['InChI'] != row['InChI']:
        pre14_inactives.at[index, 'InChI'] = smi_to_inchi(row[smi_col])
        count += 1
print(f'Updated {count} InChI values in pre14_inactives')

Updated 0 InChI values in pre14_hits


[02:07:04] WARNING: Charges were rearranged

[02:07:04] WARNING: Omitted undefined stereo

[02:07:04] WARNING: Omitted undefined stereo

[02:07:04] WARNING: Omitted undefined stereo

[02:07:04] WARNING: Omitted undefined stereo

[02:07:05] WARNING: Omitted undefined stereo



Updated 25 InChI values in pre14_inactives


[02:07:05] WARNING: Omitted undefined stereo

[02:07:05] WARNING: Omitted undefined stereo



## 14.2 Handle duplicates

In [ ]:
#Check if a mol in active set appeared in inactive set:
for i in pre14_actives[smi_col]:
    if i in list(pre14_inactives[smi_col]):
        print(f'{i} SMILES appeared in both active and inactive sets')
for i in pre14_actives['InChI']:
    if i in list(pre14_inactives['InChI']):
        print(f'{i} InChI appeared in both active and inactive sets')

#Return all duplicates by comparing InChI:
final_actives_duplicates_InChI = pre14_actives[pre14_actives.duplicated(subset=['InChI'], keep=False)]
final_inactives_duplicates_InChI = pre14_inactives[pre14_inactives.duplicated(subset=['InChI'], keep=False)]
final_actives_duplicates_smi = pre14_actives[pre14_actives.duplicated(subset=[smi_col], keep=False)]
final_inactives_duplicates_smi = pre14_inactives[pre14_inactives.duplicated(subset=[smi_col], keep=False)]

print('Number of InChI duplicates in hits: ', len(final_actives_duplicates_InChI))
print('Number of InChI duplicates in inactives: ', len(final_inactives_duplicates_InChI))
print('Number of SMILES duplicates in hits: ', len(final_actives_duplicates_smi))
print('Number of SMILES duplicates in inactives: ', len(final_inactives_duplicates_smi))

Number of InChI duplicates in hits:  0
Number of InChI duplicates in inactives:  22
Number of SMILES duplicates in hits:  0
Number of SMILES duplicates in inactives:  22


In [ ]:
if not os.path.exists(f'{data_folder}/before_finished/step_14'):
    os.makedirs(f'{data_folder}/before_finished/step_14')

#write all the duplicates to a file:
#write duplicates to a txt file: 
with open(f'{data_folder}/before_finished/step_14/duplicates.txt', 'w') as f:
    f.write('InChI duplicates in actives: \n')
    f.write(final_actives_duplicates_InChI.to_string())
    f.write('\n\n')
    f.write('InChI duplicates in inactives: \n')
    f.write(final_inactives_duplicates_InChI.to_string())
    f.write('\n\n')
    f.write('SMILES duplicates in actives: \n')
    f.write(final_actives_duplicates_smi.to_string())
    f.write('\n\n')
    f.write('SMILES duplicates in inactives: \n')
    f.write(final_inactives_duplicates_smi.to_string())

In [ ]:
#remove these duplicates, keep the first one: 
#by inchi:
final_actives = pre14_actives.drop_duplicates(subset=['InChI'], keep='first')
final_inactives = pre14_inactives.drop_duplicates(subset=['InChI'], keep='first')

In [ ]:
if len(final_actives[final_actives.duplicated(subset=[smi_col], keep=False)]) == 0:
    print('No more duplicates in hits')

if len(final_inactives[final_inactives.duplicated(subset=[smi_col], keep=False)]) == 0:
    print('No more duplicates in inactives')

No more duplicates in hits
No more duplicates in inactives


In [ ]:
# save: 
if not os.path.exists(f'{data_folder}/finished'):
    os.makedirs(f'{data_folder}/finished')
final_actives.to_csv(f'{data_folder}/finished/final_actives.csv',sep=',', index=False)
final_inactives.to_csv(f'{data_folder}/finished/final_inactives.csv',sep=',', index=False)

# Some additional modifications

## A. Adjust column names

In [ ]:
final_actives = pd.read_csv(f'{data_folder}/finished/final_actives.csv', sep=',', header=0)
final_inactives = pd.read_csv(f'{data_folder}/finished/final_inactives.csv', sep=',', header=0)

In [ ]:
# Add another column: "activity_value" with all empty NaN values for format consistency with regression datasets:
final_actives.loc[:, 'activity_value'] = np.nan 
final_inactives.loc[:, 'activity_value'] = np.nan

if 'small_organic_mol_from_mixture' not in final_actives.columns: # Sometimes, this column might be missing since no small org. mol was detected in mixtures.
    final_actives['small_organic_mol_from_mixture'] = np.nan

# Rename the columns:
final_actives = final_actives.rename(columns={
    'PUBCHEM_CID': 'CID',
    'PUBCHEM_ACTIVITY_OUTCOME': 'activity_outcome',
    'PUBCHEM_EXT_DATASOURCE_SMILES': 'SMILES',
    'Mol removed from mixture': 'mol_removed_from_mixture',
    'Small inorganic molecule': 'small_inorganic_mol_from_mixture',
    'Small organic molecule': 'small_organic_mol_from_mixture'
})
final_inactives = final_inactives.rename(columns={
    'PUBCHEM_CID': 'CID',
    'PUBCHEM_ACTIVITY_OUTCOME': 'activity_outcome',
    'PUBCHEM_EXT_DATASOURCE_SMILES': 'SMILES',
    'Mol removed from mixture': 'mol_removed_from_mixture',
    'Small inorganic molecule': 'small_inorganic_mol_from_mixture',
    'Small organic molecule': 'small_organic_mol_from_mixture'
})

#swap the positions of the columns InChI and activity_outcome:
final_actives = final_actives[['CID', 'SMILES', 'InChI', 'activity_outcome', 'activity_value', 'mol_removed_from_mixture', 'small_inorganic_mol_from_mixture', 'small_organic_mol_from_mixture']]
final_inactives = final_inactives[['CID', 'SMILES', 'InChI', 'activity_outcome', 'activity_value', 'mol_removed_from_mixture', 'small_inorganic_mol_from_mixture', 'small_organic_mol_from_mixture']]

In [ ]:
#export:
final_actives.to_csv(f'{data_folder}/finished/final_actives.csv', sep=',', index=False)
final_inactives.to_csv(f'{data_folder}/finished/final_inactives.csv', sep=',', index=False)

## B. Compile Control Data

In [ ]:
for AID in AIDs:
    exec(f"raw{AID} = pd.read_csv(f'{data_folder}/before_finished/step_1/AID{AID}.csv', sep=',', header=0)")

In [ ]:
#import inchi:
for AID in AIDs:
    exec(f"std_inchi{AID} = pd.read_csv(f'{data_folder}/before_finished/step_3/std_inchi_{AID}.txt', sep='\t', header=None)")

In [ ]:
#Update inchi
for AID in AIDs:
    exec(f"""
raw_inchi_dict{AID} = dict(zip(std_inchi{AID}[0], std_inchi{AID}[1]))
raw{AID}['InChI'] = raw{AID}['PUBCHEM_CID'].map(raw_inchi_dict{AID})
""")

In [ ]:
raw_hits = raw628[raw628['PUBCHEM_ACTIVITY_OUTCOME'] == 'Active']
raw_inactives = raw628[raw628['PUBCHEM_ACTIVITY_OUTCOME'] == 'Inactive']

In [ ]:
# Add some other columns to match the format of the curated data:
raw_hits.loc[:, 'activity_value'] = np.nan
raw_hits.loc[:, 'mol_removed_from_mixture'] = np.nan
raw_hits.loc[:, 'small_inorganic_mol_from_mixture'] = np.nan
raw_hits.loc[:, 'small_organic_mol_from_mixture'] = np.nan

raw_inactives.loc[:, 'activity_value'] = np.nan
raw_inactives.loc[:, 'mol_removed_from_mixture'] = np.nan
raw_inactives.loc[:, 'small_inorganic_mol_from_mixture'] = np.nan
raw_inactives.loc[:, 'small_organic_mol_from_mixture'] = np.nan

In [ ]:
# Rename the columns:
raw_hits = raw_hits.rename(columns={
    'PUBCHEM_CID': 'CID',
    'PUBCHEM_ACTIVITY_OUTCOME': 'activity_outcome',
    'PUBCHEM_EXT_DATASOURCE_SMILES': 'SMILES',
})
raw_inactives = raw_inactives.rename(columns={
    'PUBCHEM_CID': 'CID',
    'PUBCHEM_ACTIVITY_OUTCOME': 'activity_outcome',
    'PUBCHEM_EXT_DATASOURCE_SMILES': 'SMILES',
})

In [ ]:
#swap the positions of the columns InChI and activity_outcome:
raw_hits = raw_hits[['CID', 'SMILES', 'InChI', 'activity_outcome', 'activity_value', 'mol_removed_from_mixture', 'small_inorganic_mol_from_mixture', 'small_organic_mol_from_mixture']]
raw_inactives = raw_inactives[['CID', 'SMILES', 'InChI', 'activity_outcome', 'activity_value', 'mol_removed_from_mixture', 'small_inorganic_mol_from_mixture', 'small_organic_mol_from_mixture']]

In [ ]:
if not os.path.exists(f'{data_folder}/finished/control_data'):
    os.makedirs(f'{data_folder}/finished/control_data')

#save the hits and inactives
raw_hits.to_csv(f'{data_folder}/finished/control_data/raw_hits.csv', sep=',', index=False)
raw_inactives.to_csv(f'{data_folder}/finished/control_data/raw_inactives.csv', sep=',', index=False)

#save as txt:
raw_hits.to_csv(f'{data_folder}/finished/control_data/raw_hits.txt', sep=';', index=False, header=False)
raw_inactives.to_csv(f'{data_folder}/finished/control_data/raw_inactives.txt', sep=';', index=False, header=False)